In [1]:
#open txt file
with open ('/Users/huopeiyang/Desktop/Calvino, Italo - Invisible Cities (transl. William Weaver) (2012) - libgen.li.txt') as in_file:
    text=in_file.read()
with open ('/Users/huopeiyang/Desktop/invisible cities_ header of each chapter.txt') as in_file:
    header=in_file.read()

In [2]:
import re
regex = r"_(.+?)__"
header_each=re.findall(regex, header,re.DOTALL)

In [3]:
import re
regex = r"\n[A-Z]+[\s&]*[A-Z\s]+\s*[0-9]\s*\n\n(.+?)\n\n\n"
content_each=re.findall(regex, text)

In [4]:
content_each.extend(header_each)

In [9]:
#import reviews

In [10]:
import pandas as pd
df=pd.read_csv('/Users/huopeiyang/Desktop/thesis/selected_reviews.csv')
review=df['review'].tolist()
timestamps=df['post_time'].tolist()

In [11]:
#remove short or space 
new_review=[]
for r in review:
    new=tokenize_comments_to_sentences(r)
    for n in new:
        if len(n.split(' '))>5:
            new_review.append(n)

In [12]:
#find quotation and move them from sentences

In [13]:
import re

regex = r"“(.+?)”\s*"

def remove_quotes(comment):
    return re.sub(regex, '', comment)

comments_without_quotes = [remove_quotes(r) for r in review]


In [14]:
import re

regex = r"“(.+?)”\s*"

def find_quotes(comment):
    return re.findall(regex, comment)
quotes = [find_quotes(r) for r in review]

In [15]:
review_year=df['year'].tolist()

In [16]:
quote_year=[]
for index,q in enumerate(quotes):
    if len(q)!=0:
        quote_year.append((q,review_year[index])) 
        

In [19]:
len(comments_without_quotes)

4020

In [20]:
timestamps=df['post_time'].tolist()

In [21]:
r_t=[]
for index,r in enumerate(comments_without_quotes):
    r_t.append((r,timestamps[index]))

In [22]:
new_review_cleaned=[]
for r in comments_without_quotes:
    new=tokenize_comments_to_sentences(r)
    for n in new:
        if len(n.split(' '))>5:
            new_review_cleaned.append(n)

### create sentence data for BERTopic modeling

In [23]:
sentences = []
sentence_timestamps = []
sen_ts=[]

# Iterate over each comment and its corresponding timestamp
for comment, timestamp in r_t:
    # Split the comment into sentences
    comment_sentences = tokenize_comments_to_sentences(comment)  # This is a simple way to split sentences, but you might need a more advanced method depending on your data
    # Append the sentences and the corresponding timestamp to the lists
    for sentence in comment_sentences:
        if len(sentence.split(' '))>5:
            sentences.append(sentence)
            sentence_timestamps.extend([timestamp])
            sen_ts.append((sentence,timestamp))


In [24]:
#remove html mark

In [25]:
import re

def replace_chars(text):
    return re.sub(r'&amp;|\n|\t', ' ', text)

new_comments_html = [replace_chars(comment) for comment in new_review_cleaned]


In [53]:
#applying BERTopic

In [26]:
import pandas as pd 
from bertopic import BERTopic 
from sentence_transformers import SentenceTransformer, util 
from umap import UMAP 
from hdbscan import HDBSCAN 
from sklearn.feature_extraction.text import CountVectorizer 
from bertopic.vectorizers import ClassTfidfTransformer 
import numpy as np 
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer#'descriptions','described','describing','describe',



custom_stopwords=['travels','literature','italo','travel','calvinos','calvin','calvino','reading','read','novel','books','book','venice','invisible','polo','marco','khan','polos']
all_stopwords=list(ENGLISH_STOP_WORDS)+ custom_stopwords
vectorizer_model = CountVectorizer(stop_words= all_stopwords, ngram_range=(1, 3)) 

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model = KeyBERTInspired()
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(new_comments_html, show_progress_bar=True)
umap_model = UMAP(n_neighbors=15, n_components=5, random_state=42,
                  min_dist=0.0, metric='cosine')

topic_model = BERTopic(vectorizer_model=vectorizer_model,
                      ctfidf_model=ctfidf_model,
                      representation_model=representation_model,
                      embedding_model=sentence_model,
                      min_topic_size=20,
                      umap_model=umap_model)

topics, probs = topic_model.fit_transform(new_comments_html)
topic_model.get_topic_info()

/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/d

Batches:   0%|          | 0/565 [00:00<?, ?it/s]

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7821,-1_city_streets_inhabitants_poetic,"[city, streets, inhabitants, poetic, place, im...","[ And the mind refuses to accept more faces, m..."
1,0,873,0_prose_novels_favorite authors_fables,"[prose, novels, favorite authors, fables, fict...","[Imagine, if you will, a swirling, dynamic mix..."
2,1,663,1_felt like_glad did_patience_did really,"[felt like, glad did, patience, did really, di...",[ It is becoming irreparably squashed and torn...
3,2,625,2_visible cities_cities story_hidden cities_ci...,"[visible cities, cities story, hidden cities, ...",[Absolutely failing to even piece together a f...
4,3,523,3_kublai describing cities_mongol emperor kubl...,"[kublai describing cities, mongol emperor kubl...",[ back then i thought this was a beautifully w...
...,...,...,...,...,...
99,98,21,98_cities lost little_cities stay lost_cities ...,"[cities lost little, cities stay lost, cities ...","[ Or perhaps, speaking of other cities, I have..."
100,99,21,99_city traveller finds_new city traveller_cit...,"[city traveller finds, new city traveller, cit...","[""Arriving at each new city, the traveler find..."
101,100,21,100_men dreamed chasing_men dreams chasing_pat...,"[men dreamed chasing, men dreams chasing, path...","[ They laid the streets, each following his pu..."
102,101,20,101_true exist hypothesis_exist hypothesis tru...,"[true exist hypothesis, exist hypothesis true,...",[ So the other hypothesis is true: they exist ...


### reading each cluster and summarize them

In [672]:
number=102
topic=number+1
representative=topic_model.get_topic_info()['Representative_Docs'].tolist()
representative_word=topic_model.get_topic_info()['Representation'].tolist()

word=representative_word[topic]
doc=representative[topic]

print(word)
print (doc)
topic_model.get_topic(number)

['imaginary city uncovering', 'illusory metropolises mind', 'possible forms city', 'truth illusory metropolises', 'says city explained', 'illusory metropolises', 'rome keeping', 'imagination recreating details', 'forms city talking', 'know maybe city']
[' It may be that he is creating them all out of his imagination, or perhaps he is recreating details of his native Venice over and over again, or perhaps he is simply recounting some of the countless possible forms a city might take', ' Next week he would be in his beloved Venice, dreaming up the world, a world more real than reality, with all the ingredients needed to construct a city - memories, desires, signs, skies, trade, eyes, sounds, shapes, names and the dead', ' Is he talking about one city, Venice? Is he making it up? What is he telling us about the world, about people and their ways of creating and living in the world? And yet, what amazing cities he has conceived: doubles of each other, lost pasts, deconstructed walls, canal

[('imaginary city uncovering', 0.5024779),
 ('illusory metropolises mind', 0.40253592),
 ('possible forms city', 0.3903371),
 ('truth illusory metropolises', 0.38869002),
 ('says city explained', 0.38325673),
 ('illusory metropolises', 0.3665496),
 ('rome keeping', 0.356018),
 ('imagination recreating details', 0.35419402),
 ('forms city talking', 0.35295182),
 ('know maybe city', 0.34024963)]

In [30]:
label={0:'genre',1:'structure',2:'personal reflections',3:'structure',4:'personal reflections',5:'structure',6:'personal reflections',7:'structure',8:'personal reflections',9:'quote',10:'genre',
       11:'recommend',12:'recommend',13:'genre',14:'personal reflections',15:'genre',16:'recommend',17:'genre',18:'Venice',19:'structure',20:'genre',
      21:'structure',22:'genre',23:'personal reflections',24:'recommend',25:'genre',26:'quote',27:'recommend',28:'genre',29:'structure',30:'recommend',
       31:'structure',32:'Venice',33:'personal reflections',34:'recommend',35:'personal reflections',36:'personal reflections',37:'genre',38:'recommend',39:'quote',40:'structure',
       41:'genre',42:'recommend',43:'weird',44:'genre',45:'structure',46:'genre',47:'personal reflections',48:'recommend',49:"If On A Winter's Night a Traveler",50:'quote',
       51:'quote',52:'structure',53:'quote',54:'personal reflections',55:'recommend',56:'recommend',57:'quote',58:'genre',59:'genre',60:'structure',
       61:'personal reflections',62:'recommend',63:'quote',64:'quote',65:'weird',66:'quote',67:'quote',68:'structure',69:'quote',70:'structure',
       71:'genre',72:"If On A Winter's Night a Traveler",73:'genre',74:'quote',75:'quote',76:'genre',77:'recommend',78:'quote',79:'genre',80:'recommend',
       81:'personal reflections',82:'quote',83:'recommend',84:'recommend',85:'recommend',86:'quote',87:'recommend',88:'quote',89:'recommend',90:'genre',
       91:'quote',92:'weird',93:'personal reflections',94:'recommend',95:'quote',96:'quote',97:'weird',98:'quote',99:'quote',100:'quote',
       101:'quote',102:'Venice'
       }

In [32]:
from collections import Counter
value_frequencies = Counter(label.values())

# Print the frequencies
for value, frequency in value_frequencies.items():
    print(f"{value}: {frequency}")

genre: 20
structure: 14
personal reflections: 14
quote: 25
recommend: 21
Venice: 3
weird: 4
If On A Winter's Night a Traveler: 2


In [54]:
#reduce outlier

In [33]:
new_topics = topic_model.reduce_outliers(new_comments_html, topics)

In [34]:
topic_model.update_topics(new_comments_html, topics=new_topics)

In [35]:
topic_df=topic_model.get_topic_info()

In [36]:
topic_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,37,-1_read_over_book_shoutout,"[read, over, book, shoutout, rec, auris, tranq...","[ And the mind refuses to accept more faces, m..."
1,0,905,0_calvino_his_to_is,"[calvino, his, to, is, read, and, this, that, ...","[Imagine, if you will, a swirling, dynamic mix..."
2,1,817,1_it_didn_but_just,"[it, didn, but, just, was, me, get, really, do...",[ It is becoming irreparably squashed and torn...
3,2,662,2_invisible_cities_is_of,"[invisible, cities, is, of, to, in, and, as, t...",[Absolutely failing to even piece together a f...
4,3,659,3_kublai_marco_khan_polo,"[kublai, marco, khan, polo, visited, cities, h...",[ back then i thought this was a beautifully w...
...,...,...,...,...,...
99,98,195,98_little_lost_speaking_already,"[little, lost, speaking, already, bit, perhaps...","[ Or perhaps, speaking of other cities, I have..."
100,99,172,99_longer_lies_foreign_wait,"[longer, lies, foreign, wait, possess, arrivin...","[""Arriving at each new city, the traveler find..."
101,100,92,100_streets_built_men_woman,"[streets, built, men, woman, walls, dream, arc...","[ They laid the streets, each following his pu..."
102,101,139,101_exist_they_true_real,"[exist, they, true, real, hypothesis, do, norm...",[ So the other hypothesis is true: they exist ...


### Calculate the proportion of each topic 

In [37]:
column_topic=topic_df['Topic'].tolist()

In [38]:
column_count=topic_df['Count'].tolist()

In [50]:
pr_topic=[]
for index,name in label.items():
    if name=='genre':
        pr_topic.append(index)

In [51]:
value=0
for index,t in enumerate(column_topic):
    if t in pr_topic:
        value+= column_count[index]
print((value/18058)*4120)
print(value/18058)
print(value)

824.319415217632
0.20007752796544467
3613


### saperate data into themes

In [ ]:
#genre

In [111]:
genre_sent=[]
for index,t in enumerate(new_topics):
    if t in genre_topic:
        genre_sent.append((new_comments_html[index],sentence_timestamps[index]))

In [121]:
with open('genre_sents.json', 'w') as f:
    json.dump(genre_sent, f)

In [ ]:
#recommend

In [86]:
recommend_topic=[]
for index,name in label.items():
    if name=='recommend':
        recommend_topic.append(index)

In [115]:
recommend_sent=[]
for index,t in enumerate(new_topics):
    if t in recommend_topic:
        recommend_sent.append((new_comments_html[index],sentence_timestamps[index]))

In [ ]:
with open('recommend_sent.json', 'w') as f:
    json.dump(recommend_sent, f)

In [ ]:
#personal reflections

In [ ]:
personal_topic=[]
for index,name in label.items():
    if name=='personal reflections':
        recommend_topic.append(index)

In [ ]:
personal_sent=[]
for index,t in enumerate(new_topics):
    if t in personal_topic:
        personal_sent.append((new_comments_html[index],sentence_timestamps[index]))

In [ ]:
with open('personal_reflection_sents.json', 'w') as f:
    json.dump(recommend_sent, f)

### others work

In [87]:
quote_topic=[]
for index,name in label.items():
    if name=='quote':
        quote_topic.append(index)

In [ ]:
quote_sent=[]
for index,t in enumerate(new_topics):
    if t in recommend_topic:
        recommend_sent.append((new_comments_html[index],sentence_timestamps[index]))

In [99]:
structure_topic=[]
for index,name in label.items():
    if name=='structure':
        structure_topic.append(index)

In [101]:
#other topics
print(4120-851-740-773-824-701)

231


In [ ]:
#

In [ ]:
####

In [541]:
genre_topic = []
for key, value in label.items():
    if value == 'genre':
        genre_topic.append(key)


In [543]:
structure_topic = []
for key, value in label.items():
    if value == 'structure':
        structure_topic.append(key)

In [544]:
personal_topic = []
for key, value in label.items():
    if value == 'personal reflections':
        personal_topic.append(key)

In [545]:
topic_topic = []
for key, value in label.items():
    if value == 'topic':
        topic_topic.append(key)

In [546]:
quote_topic = []
for key, value in label.items():
    if value == 'quote':
        quote_topic.append(key)

In [ ]:
#

In [547]:
sentences_topic=[]
for index,topic in enumerate(new_topics):
    sentences_topic.append((new_review_cleaned[index],topic))

In [548]:
genre_sentence=[]
for sentence,topic in sentences_topic:
    if topic in genre_topic:
        genre_sentence.append(sentence)

In [551]:
# save the list to a text file
with open('genre_sentence.txt', 'w') as f:
    for item in genre_sentence:
        f.write("%s\n" % item)

In [552]:
structure_sentence=[]
for sentence,topic in sentences_topic:
    if topic in structure_topic:
        structure_sentence.append(sentence)

In [555]:
# save the list to a text file
with open('structure_sentence.txt', 'w') as f:
    for item in structure_sentence:
        f.write("%s\n" % item)

In [556]:
topic_sentence=[]
for sentence,topic in sentences_topic:
    if topic in topic_topic:
        topic_sentence.append(sentence)

In [557]:
# save the list to a text file
with open('topic_sentence.txt', 'w') as f:
    for item in topic_sentence:
        f.write("%s\n" % item)